## Helpers

в качестве модели выбрал CatBoost, так как он хорошо работает с категориальными признаками

In [10]:
!pip install catboost

In [52]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, roc_auc_score
import json
from ast import literal_eval
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

In [12]:
def parse_user_agent(ua_str):
    try:
        ua_dict = literal_eval(ua_str)
        return pd.Series({
            'browser': ua_dict.get('browser', 'unknown'),
            'browser_version': ua_dict.get('browser_version', 'unknown'),
            'os': ua_dict.get('os', 'unknown'),
            'os_version': ua_dict.get('os_version', 'unknown')
        })
    except:
        return pd.Series({
            'browser': 'unknown',
            'browser_version': 'unknown',
            'os': 'unknown',
            'os_version': 'unknown'
        })

def user_agent_opener(df: pd.DataFrame):
  ua_features = df['user_agent'].apply(parse_user_agent)
  df = pd.concat([df, ua_features], axis=1)
  return df

def merger(df: pd.DataFrame, geo_info: pd.DataFrame, referer_vectors: pd.DataFrame, df_labels=0):
  if isinstance(df_labels, pd.DataFrame):
    data = df.merge(referer_vectors, on='referer', how='left')
    data = data.merge(geo_info, on='geo_id', how='left')
    data = data.merge(df_labels, on='user_id', how='inner')

  else:
    data = df.merge(referer_vectors, on='referer', how='left')
    data = data.merge(geo_info, on='geo_id')

  return data

сразу открою referer_vectors и geo_info, так как они понадобятся и в части train и в части predict

In [13]:
referer_vectors = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vk_predict_sex/referer_vectors.csv', sep=';')
geo_info = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vk_predict_sex/geo_info.csv', sep=';')

в geo_info в столбце region_id много пропусков, поэтому я решил просто удалить этот столбец

в referer_vectors в столбце referer присутствуют дубликаты, причем componentX в этих строках одинаковы, поэтому я удаляю дубликаты по referer

In [14]:
geo_info = geo_info.drop('region_id', axis=1)
referer_vectors = referer_vectors.drop_duplicates(subset='referer')

---

## Train

In [15]:
train_labels = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vk_predict_sex/train_labels.csv', sep=';')
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vk_predict_sex/train.csv', sep=';')

в таблице train в столбце user_agent данные находятся в виде словаря, поэтому их необходимо "открыть"

для этого использую функцию user_agent_opener

In [16]:
train = user_agent_opener(train)

далее необходимо объеденить все таблицы, для получения фичей

для этого использую заранее подготовленную функцию merger

In [17]:
data = merger(train, geo_info, referer_vectors, train_labels)

выделяю категориальные и числовые фичи

In [18]:
cat_cols = ['browser','browser_version','os','os_version','country_id', 'timezone_id']
num_cols = [f'component{x}' for x in range(1, 10)]

выделяю таблицу фичей

In [24]:
data = data[['user_id'] + num_cols + cat_cols + ['target']]

In [26]:
X = data.drop(['user_id', 'target'], axis=1)

далее разбиваю данные на обучающую и валидационную часть и выделяю целевую переменную

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, data['target'], test_size=0.2)

после обучаю модель с помощью CatBoost

In [34]:
ctb = CatBoostClassifier(iterations=441, learning_rate=0.1, depth=8)
ctb.fit(X_train, y_train, cat_features=cat_cols)

0:	learn: 0.6629796	total: 1.5s	remaining: 10m 58s
1:	learn: 0.6386875	total: 3.32s	remaining: 12m 9s
2:	learn: 0.6192545	total: 5.8s	remaining: 14m 7s
3:	learn: 0.6033418	total: 8.02s	remaining: 14m 36s
4:	learn: 0.5899949	total: 9.31s	remaining: 13m 31s
5:	learn: 0.5773083	total: 10.6s	remaining: 12m 44s
6:	learn: 0.5682805	total: 11.7s	remaining: 12m 5s
7:	learn: 0.5604568	total: 13.2s	remaining: 11m 52s
8:	learn: 0.5527886	total: 15.6s	remaining: 12m 28s
9:	learn: 0.5461992	total: 17.3s	remaining: 12m 26s
10:	learn: 0.5406908	total: 20.3s	remaining: 13m 11s
11:	learn: 0.5364493	total: 22.7s	remaining: 13m 32s
12:	learn: 0.5331653	total: 24.2s	remaining: 13m 17s
13:	learn: 0.5288267	total: 25.6s	remaining: 13m
14:	learn: 0.5251911	total: 26.8s	remaining: 12m 42s
15:	learn: 0.5222840	total: 28.6s	remaining: 12m 39s
16:	learn: 0.5204244	total: 29.9s	remaining: 12m 26s
17:	learn: 0.5180588	total: 31.1s	remaining: 12m 10s
18:	learn: 0.5155464	total: 32.3s	remaining: 11m 57s
19:	learn: 0

получаю результат точных предсказаний

In [36]:
ctb.score(X_test, y_test)

np.float64(0.8073957991052246)

In [50]:
f1_score(ctb.predict(X_test), y_test)

0.8005583667771767

In [53]:
roc_auc_score(ctb.predict(X_test), y_test)

np.float64(0.8070870100992539)

---

## Predict

In [32]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vk_predict_sex/test.csv', sep=';')
test_users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vk_predict_sex/test_users.csv', sep=';')

в таблице test в столбце user_agent данные точно также "закрыты", поэтому их открываю точно также

In [33]:
test = user_agent_opener(test)

обьеденим test с geo_info и referer_vectors

In [38]:
pred_data = merger(test, geo_info, referer_vectors)

оставляем все необходимое

In [40]:
pred_data = pred_data[['user_id'] + num_cols + cat_cols]

далее делаем предикт

In [41]:
Xt = pred_data.drop('user_id', axis=1)
yt = ctb.predict(Xt)

в таблице train в столбце user_id были дубликаты, и могло случится так, что один пользователь является и мужчиной и женщиной.

я решил использовать моду, чтобы определить, кем является наш пользователь

In [42]:
pred_data['predicted_target'] = yt
tt = pred_data.groupby('user_id')['predicted_target'].agg(lambda x: x.mode()[0]).reset_index()

далее сопоставляем значения

In [44]:
test_users = test_users.merge(tt, on='user_id', how='left')

и сохраняем в csv-файл

In [45]:
test_users.to_csv('predicted_users.csv', index=False)

сохраняем модель

In [48]:
import pickle

with open('predict_sex_model.pkl', 'wb') as f:
  pickle.dump(ctb, f)